Load the necessary `modules` from `torch` and `torchvision`.

In [ ]:
import torch
import torchvision
from torch.optim import Adam, SGD
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms

Let us now import the necessary things from `torchgan`. For a detailed list of `losses`, `models` and `trainers` checkout the `torchgan documentation`.

We are loading the `SmallDCGAN` model instead of `DCGAN` this is because we will be using $32 \times 32$.

In [ ]:
from torchgan import *
from torchgan.models import SmallDCGANGenerator, SmallDCGANDiscriminator
from torchgan.losses import MinimaxGeneratorLoss, MinimaxDiscriminatorLoss,\
                            LeastSquaresDiscriminatorLoss, LeastSquaresGeneratorLoss
from torchgan.trainer import Trainer

Now that we have imported the required `functions` and `classes`, we need to write the `data loader`. In a future version of torchgan we will provide data loaders for some common datasets, but lets write that manually for now.

We use the `torchvision.datasets` to fetch `MNIST` for us. But MNIST has $28 \times 28$ dimensions, but our smallest models support $32 \times 32$. So we shall just pad the images before loading. Note that the trainer needs the `DataLoader` to work as it is supposed to.

Also donot forget to change the `root` as per your requirement.

In [ ]:
def mnist_dataloader():
    train_dataset = dsets.MNIST(root='/path/to/data', train=True,
                                transform=transforms.Compose([transforms.Pad((2, 2)),
                                                              transforms.ToTensor(),
                                                              transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))]),
                                download=True)
    train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    return train_loader

Since `MNIST` is `Grayscale`, we set the `channels` to `1`. You can experiment with the parameters like `step_channels` and `optimizer` but we found these to be good enough. Change the `device`, `checkpoints` and `recon` parameters if needed. Feed in a `dict` with all the optimizer options that can be applicable for the optimizer you are using.

In [ ]:
trainer = Trainer(SmallDCGANGenerator(out_channels=1, step_channels=16),
                  SmallDCGANDiscriminator(in_channels=1, step_channels=16),
                  Adam, Adam, LeastSquaresGeneratorLoss, LeastSquaresDiscriminatorLoss,
                  sample_size=64, epochs=20, verbose=5, device=torch.device("cuda:1"),
                  checkpoints="./model/gan", recon="./images",
                  optimizer_generator_options={"lr": 0.0002, "betas": (0.5, 0.999)},
                  optimizer_discriminator_options={"lr": 0.0002, "betas": (0.5, 0.999)})

Now send the MNIST DataLoader and watch the training either here or use `tensorboard`.

In [ ]:
trainer(mnist_dataloader())

In the following code block we use the `Minimax Loss`. This essentially demonstrates how you can seamlessly change loss function and run experiments fast with minimal code changes.

In [ ]:
trainer = Trainer(SmallDCGANGenerator(out_channels=1, step_channels=16),
                  SmallDCGANDiscriminator(in_channels=1, step_channels=16),
                  Adam, Adam, MinimaxGeneratorLoss, MinimaxDiscriminatorLoss,
                  sample_size=64, epochs=20, verbose=5, device=torch.device("cuda:1"),
                  checkpoints="./model/gan", recon="./images",
                  optimizer_generator_options={"lr": 0.0002, "betas": (0.5, 0.999)},
                  optimizer_discriminator_options={"lr": 0.0002, "betas": (0.5, 0.999)})

In [ ]:
trainer(mnist_dataloader())